# 0. Upload Pocket export file

First, let upload the `ril_export.html` file generated in https://getpocket.com/**export**

In [ ]:
from google.colab import files
from pathlib import Path

if not Path('ril_export.html').exists():
    uploaded = files.upload()

Saving ril_export.html to ril_export.html


In [ ]:
import datetime as dt

from bs4 import BeautifulSoup
import pandas as pd

OMNIVORE_API_URL = "https://api-prod.omnivore.app/api/graphql"
# Expires 19/08
# NIVORE_API_KEY = "19beaaaf-8a74-442d-9856-f02d6185e61a"
OMNIVORE_API_KEY = "00000000-0000-0000-0000-000000000000"
SCHEMA_URL = "https://raw.githubusercontent.com/omnivore-app/omnivore/c9fcbe72ddc6f40dd06e7073b8ffe3c1e71bd650/packages/api/src/generated/schema.graphql"
REQUESTS_SLEEP_TIME = 1 # Number of seconds

# 1. Parse the export file

The html has the following extructure:

- `<h1>` Unread
- `<ul>` with list items of `<a>`. The href is the link to the article, and the anchor text is the title. It also has a `tags` and `time_added` attributes.
- `<h1>` Read
- Another `<ul>` like the one above

We will transform this into tabular data, with the following fields:
- *read*: Boolean on wether the article has been read
- *time_added*: The time the item was added
- *tags*: An array of strings
- *href*: The url
- *title*: The title of the article

In [ ]:
with open('ril_export.html', 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')

soup.title

<title>Pocket Export</title>

In [ ]:
soup.findAll('h1')

[<h1>Unread</h1>, <h1>Read Archive</h1>]

In [ ]:
def process_list(h1) -> pd.DataFrame:
    ul = h1.find_next_sibling('ul')
    print(len(ul), h1.text, 'articles')
    read = h1.text != 'Unread'

    items = []
    for a in ul.findAll('a', href=True):
        items.append({
            'read': read,
            'time_added': a['time_added'],
            'href': a['href'],
            'tags': a['tags'],
            'title': a.text,
        })

    df = pd.DataFrame(items)
    df['time_added'] = pd.to_datetime(df['time_added'], unit='s')
    return df

In [ ]:
df = pd.concat([process_list(h1) for h1 in soup.findAll('h1')])
df

345 Unread articles
549 Read Archive articles


,read,time_added,href,tags,title
0,False,2023-04-21 06:01:17,http://getpocket.com/explore/item/don-t-forget...,,"Don’t Forget: You, Too, Can Acquire a Super Me..."
1,False,2023-04-06 08:48:02,https://blog.sigplan.org/2023/04/04/teaching-c...,,Teaching Category Theory to Computer Scientist...
2,False,2023-03-21 08:25:00,https://risk-engineering.org/concept/Farmer-di...,,"Farmer’s diagram, or F-N curve: Representing s..."
3,False,2023-03-21 08:24:47,http://ascii.textfiles.com/archives/5509,,"Discord, or the Death of Lore « ASCII by Jason..."
4,False,2023-03-19 14:16:19,https://www.nature.com/articles/d41586-021-000...,,https://www.nature.com/articles/d41586-021-000...
...,...,...,...,...,...
269,True,2020-08-06 10:10:50,https://www.reddit.com/r/Anki/comments/hjz0u1/...,,New Add-on: Time Spent Creating Card’s
270,True,2020-07-18 16:57:21,https://m.huffingtonpost.es/entry/ladrillo-y-m...,,https://m.huffingtonpost.es/entry/ladrillo-y-m...
271,True,2020-07-18 16:45:10,https://www.youtube.com/watch?v=FxedY4gzlJ4,,https://www.youtube.com/watch?v=FxedY4gzlJ4
272,True,2020-07-03 17:57:23,https://www.xataka.com/especiales/repensando-c...,,Repensando la carrera de ingeniería informátic...


# 2. Realize parse was not needed, just upload the file

At least using bs is fun

In [119]:
!pip install -q gql[all]

In [120]:
import requests

with requests.get(SCHEMA_URL) as r:
    r.raise_for_status()
    schema = r.text

    assert schema is not None

print(schema[:100])

directive @sanitize(allowedTags: [String], maxLength: Int, minLength: Int, pattern: String) on INPUT


In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Select your transport with a defined url endpoint

transport = RequestsHTTPTransport(
    url=OMNIVORE_API_URL,
    headers = {
        'authorization': OMNIVORE_API_KEY,
    }
)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, schema=schema, fetch_schema_from_transport=False)

In [ ]:
# Doing a "test query" to check if everything is correct
r = client.execute(gql("""
query Viewer {
    me {
        id
        name
        profile {
            username
        }
    }
}
"""))

USERNAME = r['me']['profile']['username']

print(f"Hello {r['me']['name']} ({USERNAME})!")

Hello David Davó (ddavo)!


In [ ]:
createArticle = gql("""
  mutation CreateArticleSavingRequest($url: String!) {
    createArticleSavingRequest(input: {url: $url}) {
      ... on CreateArticleSavingRequestSuccess {
        articleSavingRequest {
          id
          status
          slug
          createdAt
          updatedAt
          url
          errorCode
        }
      }
      ... on CreateArticleSavingRequestError {
        errorCodes
      }
    }
  }
""")

updatePageSavedDate =  gql("""
mutation UpdatePageDate($id: ID!, $date: Date!) {
    updatePage(input: {pageId: $id, savedAt: $date}) {
        ... on UpdatePageSuccess {
            updatedPage {
                id
                savedAt
                publishedAt
                title
            }
        }
        ...on UpdatePageError {
            errorCodes
        }
    }
}
""")

archivePage = gql("""
mutation ArchivePage($id: ID!) {
    setLinkArchived (input: {linkId: $id, archived: true}) {
        ... on ArchiveLinkSuccess {
            linkId
            message
        }
        ... on ArchiveLinkError {
            message
            errorCodes
        }
    }
}
""")

def saveArticle(url, date: dt.datetime = None, archive: bool = False):
    # First createArticleSavingRequest
    r = client.execute(createArticle, variable_values={'url': url})
    rid = r['createArticleSavingRequest']['articleSavingRequest']['id']
    rslug = r['createArticleSavingRequest']['articleSavingRequest']['slug']
    print(r)

    # Then updatePage to change the saved date
    if date is not None:
        r = client.execute(updatePageSavedDate, {
            'id': rid,
            'date': date.isoformat(),
        })
        print(r)

    # Then, if the content is already read, archive it directly
    if archive:
        r = client.execute(archivePage, {
            'id': rid
        })

saveArticle("https://blog.ddavo.me/posts/tutorials/ros2-coppelia-lidar/", dt.datetime(2023,5,1), True)

{'createArticleSavingRequest': {'articleSavingRequest': {'id': '3753f03c-e667-47de-91e2-95f315fbccdc', 'status': 'PROCESSING', 'slug': 'https-blog-ddavo-me-posts-tutorials-ros-2-coppelia-lidar-1883f1abbe3', 'createdAt': '2023-05-21T16:18:16.675Z', 'updatedAt': '2023-05-21T16:58:49.684Z', 'url': 'https://blog.ddavo.me/posts/tutorials/ros2-coppelia-lidar', 'errorCode': None}}}
{'updatePage': {'updatedPage': {'id': '3753f03c-e667-47de-91e2-95f315fbccdc', 'savedAt': '2023-05-01T00:00:00.000Z', 'publishedAt': None, 'title': 'https://blog.ddavo.me/posts/tutorials/ros2-coppelia-lidar/'}}}


In [ ]:
from tqdm import tqdm
from time import sleep

for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    saveArticle(
        url=row['href'],
        date=row['time_added'],
        archive=row['read']
    )
    sleep(REQUESTS_SLEEP_TIME)

  0%|          | 0/446 [00:00<?, ?it/s]

{'createArticleSavingRequest': {'articleSavingRequest': {'id': 'fe6413d6-22ab-490c-8786-767b510ecf7f', 'status': 'PROCESSING', 'slug': 'http-getpocket-com-explore-item-don-t-forget-you-too-can-acquire-1883f42dc7b', 'createdAt': '2023-05-21T17:02:06.459Z', 'updatedAt': '2023-05-21T17:02:44.691Z', 'url': 'http://getpocket.com/explore/item/don-t-forget-you-too-can-acquire-a-super-memory', 'errorCode': None}}}
{'updatePage': {'updatedPage': {'id': 'fe6413d6-22ab-490c-8786-767b510ecf7f', 'savedAt': '2023-04-21T06:01:17.000Z', 'publishedAt': None, 'title': 'http://getpocket.com/explore/item/don-t-forget-you-too-can-acquire-a-super-memory'}}}


  0%|          | 1/446 [00:05<41:51,  5.64s/it]

{'createArticleSavingRequest': {'articleSavingRequest': {'id': '568bb907-6451-4099-994f-0b25317e5397', 'status': 'SUCCEEDED', 'slug': 'https-blog-sigplan-org-2023-04-04-teaching-category-theory-to-co-1883f42e9cf', 'createdAt': '2023-05-21T17:02:09.871Z', 'updatedAt': '2023-05-21T17:03:33.003Z', 'url': 'https://blog.sigplan.org/2023/04/04/teaching-category-theory-to-computer-scientists', 'errorCode': None}}}
{'updatePage': {'updatedPage': {'id': '568bb907-6451-4099-994f-0b25317e5397', 'savedAt': '2023-04-06T08:48:02.000Z', 'publishedAt': '2023-04-04T13:00:46.000Z', 'title': 'Teaching Category Theory to Computer Scientists | SIGPLAN Blog'}}}


  0%|          | 2/446 [00:11<42:16,  5.71s/it]

...

100%|█████████▉| 444/446 [51:21<00:14,  7.43s/it]

{'createArticleSavingRequest': {'articleSavingRequest': {'id': 'e75df910-371f-4831-a86d-63659e8d3faf', 'status': 'PROCESSING', 'slug': 'https-www-xataka-com-especiales-repensando-carrera-ingenieria-in-1883f79df68', 'createdAt': '2023-05-21T18:02:11.000Z', 'updatedAt': '2023-05-21T18:02:12.000Z', 'url': 'https://www.xataka.com/especiales/repensando-carrera-ingenieria-informatica-hablan-profesionales', 'errorCode': None}}}
{'updatePage': {'updatedPage': {'id': 'e75df910-371f-4831-a86d-63659e8d3faf', 'savedAt': '2020-07-03T17:57:23.000Z', 'publishedAt': None, 'title': 'https://www.xataka.com/especiales/repensando-carrera-ingenieria-informatica-hablan-profesionales'}}}


100%|█████████▉| 445/446 [51:28<00:07,  7.05s/it]

{'createArticleSavingRequest': {'articleSavingRequest': {'id': 'a2d488e2-469a-4e20-814d-acb7253694f4', 'status': 'PROCESSING', 'slug': 'https-medium-com-pocket-tu-experiencia-en-pocket-comienza-ahora--1883f79f75b', 'createdAt': '2023-05-21T18:02:17.000Z', 'updatedAt': '2023-05-21T18:02:18.000Z', 'url': 'https://medium.com/@Pocket/tu-experiencia-en-pocket-comienza-ahora-aprov%C3%A9chala-al-m%C3%A1ximo-b8bb4a065c95', 'errorCode': None}}}
{'updatePage': {'updatedPage': {'id': 'a2d488e2-469a-4e20-814d-acb7253694f4', 'savedAt': '2020-06-28T07:51:37.000Z', 'publishedAt': None, 'title': 'https://medium.com/@Pocket/tu-experiencia-en-pocket-comienza-ahora-aprov%C3%A9chala-al-m%C3%A1ximo-b8bb4a065c95'}}}


100%|██████████| 446/446 [51:33<00:00,  6.94s/it]
